# Pythia-70M with Block-Sparse Attention - Complete Demo

## Deep Learning Systems Project

This notebook demonstrates:
1. Model architecture and creation
2. Block-sparse attention patterns
3. Training with HuggingFace datasets
4. Performance benchmarking
5. Model checkpointing and loading
6. Text generation

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('./python')
sys.path.append('./apps')

import time
import numpy as np
import matplotlib.pyplot as plt
import needle as ndl
import needle.nn as nn

from pythia_model import create_pythia_70m, PythiaConfig
from train_pythia import (
    train, load_dataset_huggingface, batchify, 
    save_checkpoint, load_checkpoint
)
from needle.nn.nn_sparse_attention import BlockSparsePattern

print("✓ All imports successful!")
print(f"Needle backend: {ndl.backend_selection.BACKEND}")

## 2. Model Creation

In [ ]:
# Configuration
device = ndl.cpu()
vocab_size = 10000
max_seq_len = 256

print("Creating Pythia-70M Models...\n")

# Dense model
print("1. Dense Attention Model")
model_dense, config_dense = create_pythia_70m(
    vocab_size=vocab_size,
    max_seq_len=max_seq_len,
    use_sparse_attention=False,
    device=device
)

# Sparse model
print("\n2. Sparse Attention Model")
model_sparse, config_sparse = create_pythia_70m(
    vocab_size=vocab_size,
    max_seq_len=max_seq_len,
    use_sparse_attention=True,
    device=device
)

print("\n" + "="*80)
print("Model Specifications:")
print(f"  Total Parameters: ~{config_dense.get_total_params() / 1e6:.1f}M")
print(f"  Layers: {config_dense.num_layers}")
print(f"  Hidden Dimension: {config_dense.d_model}")
print(f"  Attention Heads: {config_dense.num_heads}")
print(f"  FFN Dimension: {config_dense.d_ff}")
print(f"  Vocabulary Size: {vocab_size}")
print(f"  Max Sequence Length: {max_seq_len}")
print("="*80)

## 3. Block-Sparse Attention Patterns

In [ ]:
seq_len = 256
block_size = 64

print(f"Sequence length: {seq_len}")
print(f"Block size: {block_size}")
print(f"Number of blocks: {seq_len // block_size}\n")

# Generate patterns
local_pattern = BlockSparsePattern.local_pattern(seq_len, block_size, window_size=1)
global_pattern = BlockSparsePattern.global_pattern(seq_len, block_size, stride=2)
mixed_pattern = BlockSparsePattern.mixed_pattern(seq_len, block_size, window_size=1, stride=4)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].imshow(local_pattern, cmap='Blues', aspect='auto')
axes[0].set_title('Local Pattern (Sliding Window)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Key Blocks')
axes[0].set_ylabel('Query Blocks')
axes[0].grid(False)

axes[1].imshow(global_pattern, cmap='Greens', aspect='auto')
axes[1].set_title('Global Pattern (Strided)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Key Blocks')
axes[1].set_ylabel('Query Blocks')
axes[1].grid(False)

axes[2].imshow(mixed_pattern, cmap='Reds', aspect='auto')
axes[2].set_title('Mixed Pattern (Local + Global)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Key Blocks')
axes[2].set_ylabel('Query Blocks')
axes[2].grid(False)

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/sparse_patterns_notebook.png', dpi=150, bbox_inches='tight')
plt.show()

# Calculate sparsity
print("\nSparsity Analysis:")
print(f"  Local pattern: {(1 - local_pattern.sum() / local_pattern.size) * 100:.1f}% sparse")
print(f"  Global pattern: {(1 - global_pattern.sum() / global_pattern.size) * 100:.1f}% sparse")
print(f"  Mixed pattern: {(1 - mixed_pattern.sum() / mixed_pattern.size) * 100:.1f}% sparse")

## 4. Forward Pass Testing

In [ ]:
batch_size = 8
seq_len = 128

# Create random input
input_ids = ndl.Tensor(
    np.random.randint(0, vocab_size, (batch_size, seq_len)),
    device=device
)

print(f"Input shape: {input_ids.shape}\n")
print("="*80)

# Dense forward pass
print("Dense Attention Forward Pass...")
times_dense = []
for i in range(5):
    start = time.time()
    logits_dense, _ = model_dense(input_ids)
    elapsed = time.time() - start
    times_dense.append(elapsed)
    print(f"  Run {i+1}: {elapsed:.4f}s")

avg_time_dense = np.mean(times_dense[1:])
print(f"\nAverage (excluding warmup): {avg_time_dense:.4f}s")
print(f"Output shape: {logits_dense.shape}")

print("\n" + "="*80)

# Sparse forward pass
print("Sparse Attention Forward Pass...")
times_sparse = []
for i in range(5):
    start = time.time()
    logits_sparse, _ = model_sparse(input_ids)
    elapsed = time.time() - start
    times_sparse.append(elapsed)
    print(f"  Run {i+1}: {elapsed:.4f}s")

avg_time_sparse = np.mean(times_sparse[1:])
print(f"\nAverage (excluding warmup): {avg_time_sparse:.4f}s")
print(f"Output shape: {logits_sparse.shape}")

speedup = avg_time_dense / avg_time_sparse

print("\n" + "="*80)
print("Performance Summary:")
print(f"  Dense: {avg_time_dense:.4f}s")
print(f"  Sparse: {avg_time_sparse:.4f}s")
print(f"  Speedup: {speedup:.2f}×")
print("="*80)

## 5. Dataset Loading (HuggingFace)

In [ ]:
# Try to load WikiText-2
print("Loading WikiText-2 dataset...\n")

try:
    train_data, val_data, vocab_size = load_dataset_huggingface(
        "wikitext-2", 
        max_tokens=50000  # Small for demo
    )
    
    print(f"\n✓ Dataset loaded successfully!")
    print(f"  Train tokens: {len(train_data):,}")
    print(f"  Val tokens: {len(val_data):,}")
    print(f"  Vocabulary size: {vocab_size:,}")
    
    # Show sample
    print(f"\nSample token indices: {train_data[:20]}")
    
except Exception as e:
    print(f"Note: {e}")
    print("Using synthetic data for demo...")
    vocab_size = 10000
    train_data = np.random.randint(0, vocab_size, size=50000)
    val_data = np.random.randint(0, vocab_size, size=5000)

## 6. Training Demo (Short)

In [ ]:
# Short training demo
print("Running short training demo...\n")

# Recreate smaller model for faster training
model_demo, config_demo = create_pythia_70m(
    vocab_size=vocab_size,
    max_seq_len=64,  # Shorter for speed
    use_sparse_attention=True,
    device=device
)

# Train for just 2 epochs
results = train(
    model=model_demo,
    train_data=train_data,
    val_data=val_data,
    config=config_demo,
    n_epochs=2,
    batch_size=16,
    seq_len=64,
    lr=1e-3,
    device=device,
    checkpoint_dir='./demo_checkpoints'
)

print("\n✓ Training demo complete!")

## 7. Model Checkpointing

In [ ]:
print("Demonstrating model checkpointing...\n")

# Save checkpoint
optimizer = ndl.optim.Adam(model_demo.parameters(), lr=1e-3)
save_checkpoint(
    model_demo, 
    optimizer, 
    epoch=2, 
    loss=results['train_losses'][-1],
    filepath='/mnt/user-data/outputs/demo_checkpoint.pkl'
)

print("\nLoading checkpoint back...")
loaded_model, loaded_opt, epoch, loss = load_checkpoint(
    '/mnt/user-data/outputs/demo_checkpoint.pkl',
    device
)

print("\n✓ Checkpoint save/load successful!")

## 8. Training Curves

In [ ]:
if 'train_losses' in results:
    epochs = range(1, len(results['train_losses']) + 1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Training loss
    ax1.plot(epochs, results['train_losses'], 'b-o', label='Training Loss', linewidth=2)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Validation loss
    ax2.plot(epochs, results['val_losses'], 'r-s', label='Validation Loss', linewidth=2)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Loss', fontsize=12)
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('/mnt/user-data/outputs/training_curves_notebook.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nFinal Results:")
    print(f"  Final Train Loss: {results['train_losses'][-1]:.4f}")
    print(f"  Final Val Loss: {results['val_losses'][-1]:.4f}")
else:
    print("No training results available yet")

## 9. Complexity Analysis

In [ ]:
seq_lengths = [64, 128, 256, 512, 1024]
d_model = 512
n_heads = 8
n_layers = 6
block_size = 64
sparsity = 0.75

dense_flops = []
sparse_flops = []

for seq_len in seq_lengths:
    # Dense: O(n²d)
    dense_ops = n_layers * seq_len * seq_len * d_model
    # Sparse: O(n * block * d * (1 - sparsity))
    sparse_ops = n_layers * seq_len * block_size * d_model * (1 - sparsity)
    
    dense_flops.append(dense_ops / 1e9)
    sparse_flops.append(sparse_ops / 1e9)

# Plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(seq_lengths, dense_flops, 'b-o', label='Dense', linewidth=2, markersize=8)
plt.plot(seq_lengths, sparse_flops, 'r-s', label='Sparse (75%)', linewidth=2, markersize=8)
plt.xlabel('Sequence Length', fontsize=12)
plt.ylabel('GFLOPs', fontsize=12)
plt.title('Computational Complexity', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')

plt.subplot(1, 2, 2)
speedups = [d/s for d, s in zip(dense_flops, sparse_flops)]
plt.bar(range(len(seq_lengths)), speedups, color='green', alpha=0.7)
plt.xticks(range(len(seq_lengths)), seq_lengths)
plt.axhline(y=1.0, color='r', linestyle='--', linewidth=2, label='Baseline')
plt.xlabel('Sequence Length', fontsize=12)
plt.ylabel('Theoretical Speedup (×)', fontsize=12)
plt.title('Sparse Attention Speedup', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, v in enumerate(speedups):
    plt.text(i, v + 0.1, f'{v:.2f}×', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/complexity_analysis_notebook.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nTheoretical Speedup:")
print(f"{'Seq Len':<12} {'Dense GFLOPs':<15} {'Sparse GFLOPs':<15} {'Speedup':<10}")
print("-"*60)
for i, seq_len in enumerate(seq_lengths):
    print(f"{seq_len:<12} {dense_flops[i]:<15.2f} {sparse_flops[i]:<15.2f} {speedups[i]:<10.2f}×")

## 10. Text Generation Demo

In [ ]:
print("Text Generation Demo\n")
print("="*80)

# Create prompt
prompt = ndl.Tensor(
    np.array([[1, 2, 3, 4, 5]]),
    device=device
)

print(f"Prompt tokens: {prompt.numpy()[0]}")
print(f"\nGenerating 15 tokens with sparse model...")

model_sparse.eval()
generated = model_sparse.generate(
    prompt, 
    max_new_tokens=15, 
    temperature=1.0
)

generated_tokens = generated.numpy()[0]
print(f"Generated: {generated_tokens}")

print("\nNote: This is a demonstration with an untrained model.")
print("For meaningful text, train the model on a real dataset first.")
print("="*80)

## 11. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("PROJECT SUMMARY")
print("="*80)

print("\n✓ Completed Demonstrations:")
print("  1. Model creation (dense and sparse)")
print("  2. Sparse attention pattern visualization")
print("  3. Forward pass performance comparison")
print("  4. HuggingFace dataset integration")
print("  5. Training pipeline")
print("  6. Model checkpointing and loading")
print("  7. Training curve visualization")
print("  8. Complexity analysis")
print("  9. Text generation")

print("\n📊 Key Results:")
print(f"  • Model Size: ~70M parameters")
print(f"  • Forward Pass Speedup: {speedup:.2f}×")
print(f"  • Sparsity: ~75% for local pattern")
print(f"  • Theoretical Speedup (1024 tokens): {speedups[-1]:.2f}×")

print("\n🚀 Next Steps:")
print("  1. Train on full WikiText-2 dataset")
print("  2. Run comprehensive benchmarks")
print("  3. Experiment with different sparse patterns")
print("  4. Scale to larger models")
print("  5. Optimize CUDA kernels")

print("\n📚 References:")
print("  • Pythia (Biderman et al., 2023)")
print("  • Sparse Transformer (Child et al., 2019)")
print("  • Longformer (Beltagy et al., 2020)")
print("  • BigBird (Zaheer et al., 2020)")

print("\n" + "="*80)
print("DEMO COMPLETE!")
print("="*80)